# 新一线城市可视化

In [1]:
# 导入包
import numpy as np 
import pandas as pd

import plotly as py 
import plotly.graph_objs as go 
import plotly.express as px 
from plotly import tools

## 房价数据可视化

1. 房价走势图 - 条形图
2. 人口走势图 - 折线图
3. 人口增量走势图 - 折线图
4. GDP走势图 - 折线图
5. 人口 Vs GDP随时间变化走势图 - 散点图 

In [2]:
# 读入数据
df_price = pd.read_excel('../data/新一线城市房价数据.xlsx')
df_price.head() 

dates    price name
0  2011年12月房价  8477元/㎡   成都
1  2011年11月房价  8562元/㎡   成都
2  2011年10月房价  8637元/㎡   成都
3  2011年09月房价  8679元/㎡   成都
4  2011年08月房价  8720元/㎡   成都

In [3]:
# 提取年月
df_price['dates'] = df_price['dates'].str.replace('房价', '') 
# 提取年份
df_price['year'] = df_price['dates'].str.split('年').str[0]

# price处理
df_price['price'] = df_price['price'].str.replace('元/㎡', '').astype('int')

# 排序
df_price = df_price.sort_values('dates')
df_price.head() 

dates  price name  year
701   2011年01月   9058   苏州  2011
1382  2011年01月  15243   青岛  2011
1605  2011年01月   6106   佛山  2011
471   2011年01月   6984   西安  2011
931   2011年01月   5920   郑州  2011

In [4]:
# 生成新表
df_price2 = df_price.groupby(['name', 'year'])['price'].mean().reset_index()
df_price2['price'] = round(df_price2['price'])
# 排序
df_price2 = df_price2.sort_values(['year', 'price'])
df_price2.head() 

name  year   price
10    佛山  2011  6282.0
110   郑州  2011  6421.0
30    合肥  2011  6539.0
130   长沙  2011  6558.0
0     东莞  2011  6640.0

In [16]:
# 条形图
fig1 = px.bar(df_price2, x='name', y='price', color='name', text='price',
             title='新一线城市近10年平均房价走势图', 
             animation_frame='year',
             range_y=[5000, 30000]
             ) 
fig1.update_layout(xaxis_title='城市', yaxis_title='平均价格(元/㎡)')
py.offline.plot(fig1, filename='新一线城市近10年平均房价走势图.html') 

'新一线城市近10年平均房价走势图.html'

## 人口-GDP数据可视化

In [5]:
# 读入数据
df_gdp = pd.read_excel('../data/新一线城市人口和GDP.xlsx', sheet_name=0)
df_pop = pd.read_excel('../data/新一线城市人口和GDP.xlsx', sheet_name=1)
print(df_gdp.shape, df_pop.shape) 

(300, 3) (300, 3)


In [6]:
# 合并数据
df_all = pd.merge(left=df_gdp, right=df_pop, on=['城市', '年份']) 
df_all = df_all.sort_values(['城市','年份']) 
df_all.head() 

城市    年份      GDP      人口
219  东莞  2000   489.73  152.61
218  东莞  2001   578.93  153.89
217  东莞  2002   672.89  156.19
216  东莞  2003   947.97  158.96
215  东莞  2004  1155.30  161.97

In [19]:
# 条形图
fig2 = px.bar(df_all, x='城市', y='GDP', color='城市', text='GDP', 
             title='新一线城市近20年GDP变化趋势',
             range_y=[300, 25000],
             animation_frame='年份',
             ) 
fig2.update_layout(yaxis_title='GDP(亿元)') 
py.offline.plot(fig2, filename='2000-2019年GDP变化趋势.html')

'2000-2019年GDP变化趋势.html'

In [20]:
# 散点图
fig3 = px.scatter(df_all, x='GDP', y='人口', animation_frame='年份', animation_group='城市', 
                 size='人口', color='城市', hover_name='城市', size_max=50, text='城市',
                 range_x=[300, 25000], range_y=[150, 4000],
                 title='新一线城市近20年GDP和人口变化趋势', 
                ) 
fig3.update_layout(xaxis_title='GDP(亿元)', yaxis_title='人口(万人)')
py.offline.plot(fig3, filename='2000-2019年GDP和人口变化趋势.html')

'2000-2019年GDP和人口变化趋势.html'

## 人口增量数据可视化

In [46]:
# 存储数据
df_dealed = pd.DataFrame() 

for i in df_all['城市'].unique():
    # 筛选城市、创建副本
    df_sel = df_all[df_all['城市']==i].copy() 
    # 新增列
    df_sel['人口增量'] = df_sel['人口'].diff() 
    # 累计增量
    df_sel['人口累计增量'] = df_sel['人口增量'].cumsum()
    # 追加
    df_dealed = df_dealed.append(df_sel, ignore_index=True) 
    
# 删除空值
df_dealed = df_dealed.dropna()
df_dealed['人口增量'] = round(df_dealed['人口增量'], 2) 
df_dealed['人口累计增量'] = round(df_dealed['人口累计增量'], 2) 
df_dealed.head() 

城市    年份      GDP      人口  人口增量  人口累计增量
1  东莞  2001   578.93  153.89  1.28    1.28
2  东莞  2002   672.89  156.19  2.30    3.58
3  东莞  2003   947.97  158.96  2.77    6.35
4  东莞  2004  1155.30  161.97  3.01    9.36
5  东莞  2005  2181.62  165.65  3.68   13.04

In [52]:
# 条形图
fig4 = px.bar(df_dealed, x='城市', y='人口累计增量', text='人口累计增量', color='城市',
               title='新一线城市2000年后人口累计增量', animation_frame='年份',
               range_y=[0, 500]
              )
fig4.update_layout(yaxis_title='累计增量(万人)') 
py.offline.plot(fig4, filename='2001-2019年人口增量数据.html') 

'2001-2019年人口增量数据.html'